In [12]:
import pandas as pd

In [13]:
df = pd.read_csv('data/test_data.csv')
df

,Page Number,Content
0,Page 1,==== C:/Users/Kassem Anis/OneDrive - Debreceni...
1,Page 2,vulnerability management is to assess each mit...
2,Page 3,==== C:/Users/Kassem Anis/OneDrive - Debreceni...
3,Page 4,==== C:/Users/Kassem Anis/OneDrive - Debreceni...
4,Page 5,open the advanced Task wizard dialog in this d...
5,Page 6,list and type Apache in the search box and sel...
6,Page 7,the openvas agrees with that assessment a seve...
7,Page 8,through these various sources of security info...
8,Page 9,related resources and articles searching the d...
9,Page 10,==== C:/Users/Kassem Anis/OneDrive - Debreceni...


In [14]:
df['Content'] = df['Content'].apply(lambda x: x.lower())
df.head()

,Page Number,Content
0,Page 1,==== c:/users/kassem anis/onedrive - debreceni...
1,Page 2,vulnerability management is to assess each mit...
2,Page 3,==== c:/users/kassem anis/onedrive - debreceni...
3,Page 4,==== c:/users/kassem anis/onedrive - debreceni...
4,Page 5,open the advanced task wizard dialog in this d...


In [15]:
import nltk
import string

nltk.download('punkt')

def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

from tqdm import tqdm

for i in tqdm(range(len(df))):
    df.loc[i, 'Content'] = remove_punctuation(df.loc[i, 'Content'])
df.head()

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/zafiraibraeva/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
100%|██████████| 16/16 [00:00<00:00, 8403.31it/s]


,Page Number,Content
0,Page 1,cuserskassem anisonedrive debreceni\negyetem...
1,Page 2,vulnerability management is to assess each mit...
2,Page 3,cuserskassem anisonedrive debreceni\negyetem...
3,Page 4,cuserskassem anisonedrive debreceni\negyetem...
4,Page 5,open the advanced task wizard dialog in this d...


In [16]:
def remove_numbers(text):
    result = ''.join([i for i in text if not i.isdigit()])
    return result

df['Content'] = df['Content'].apply(lambda x: remove_numbers(x))
df.head()

,Page Number,Content
0,Page 1,cuserskassem anisonedrive debreceni\negyetem...
1,Page 2,vulnerability management is to assess each mit...
2,Page 3,cuserskassem anisonedrive debreceni\negyetem...
3,Page 4,cuserskassem anisonedrive debreceni\negyetem...
4,Page 5,open the advanced task wizard dialog in this d...


In [17]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt_tab')

def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
    return ' '.join(filtered_sentence)
tqdm.pandas()
df['Content'] = df['Content'].progress_apply(lambda x: remove_stopwords(x))
df.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/zafiraibraeva/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/zafiraibraeva/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
100%|██████████| 16/16 [00:00<00:00, 1261.92it/s]


,Page Number,Content
0,Page 1,cuserskassem anisonedrive debreceni egyetemasz...
1,Page 2,vulnerability management assess mitigation ens...
2,Page 3,cuserskassem anisonedrive debreceni egyetemasz...
3,Page 4,cuserskassem anisonedrive debreceni egyetemasz...
4,Page 5,open advanced task wizard dialog dialogue init...


In [18]:
# from nltk.stem import PorterStemmer
# 
# def stem_text(text):
#     stemmer = PorterStemmer()
#     word_tokens = word_tokenize(text)
#     
#     stemmed_sentence = [stemmer.stem(w) for w in word_tokens]
#     return ' '.join(stemmed_sentence)
# tqdm.pandas()
# df['Content'] = df['Content'].progress_apply(lambda x: stem_text(x))
# df.head()

In [19]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    word_tokens = word_tokenize(text)
    
    lemmatized_text = [lemmatizer.lemmatize(w) for w in word_tokens]
    return ' '.join(lemmatized_text)

tqdm.pandas()
df['Content'] = df['Content'].progress_apply(lambda x: lemmatize_text(x))
df.head()

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/zafiraibraeva/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
100%|██████████| 16/16 [00:00<00:00, 1122.32it/s]


,Page Number,Content
0,Page 1,cuserskassem anisonedrive debreceni egyetemasz...
1,Page 2,vulnerability management assess mitigation ens...
2,Page 3,cuserskassem anisonedrive debreceni egyetemasz...
3,Page 4,cuserskassem anisonedrive debreceni egyetemasz...
4,Page 5,open advanced task wizard dialog dialogue init...


In [20]:
df.to_csv('data/test_data_norm.csv', index=False)

In [22]:
from collections import Counter
from keybert import KeyBERT
from tqdm import tqdm
import pandas as pd

df = pd.read_csv('data/test_data_norm.csv')

kw_model = KeyBERT('sentence-transformers/all-MiniLM-L6-v2') 

def generate_tags(text, model, num_tags=3):
    try:
        keywords = model.extract_keywords(text, keyphrase_ngram_range=(1, 2), stop_words='english', top_n=num_tags)
        return [kw[0] for kw in keywords]
    except Exception as e:
        return []

tqdm.pandas()
df['Tags'] = df['Content'].progress_apply(lambda x: generate_tags(x, kw_model, num_tags=3))

all_tags = [tag for tags in df['Tags'] for tag in tags]
top_tags = [tag for tag, count in Counter(all_tags).most_common(3)]

print("Top Tags for the Entire File:", top_tags)
df.head()

100%|██████████| 16/16 [00:04<00:00,  3.87it/s]

Top Tags for the Entire File: ['vulnerability management', 'vulnerability identification', 'vulnerability managing']


,Page Number,Content,Tags
0,Page 1,cuserskassem anisonedrive debreceni egyetemasz...,"[vulnerability managing, vulnerability managem..."
1,Page 2,vulnerability management assess mitigation ens...,"[vulnerability management, environment cyberse..."
2,Page 3,cuserskassem anisonedrive debreceni egyetemasz...,"[map utility, like netcat, information tool]"
3,Page 4,cuserskassem anisonedrive debreceni egyetemasz...,"[vulnerability identification, security manage..."
4,Page 5,open advanced task wizard dialog dialogue init...,"[discovering vulnerability, vulnerability mana..."
